In [1]:
import reg_mri
import os
from glob import glob
from utils import compute_mean_dice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import shutil
import scipy.io
import matplotlib.pyplot as plt
from tqdm import tqdm
from transforms_dict import getRegistrationEvalInverseTransformForMRI
import subprocess

In [2]:
dataset = 'dataset3/GIN_Mean'
output_dir = 'output/GIN/ANTS-Mean/'
outfolder = 'ANTS-Mean'

In [3]:
atlas = os.path.join('dataset3', 'Atlas', 'P56_Atlas_128_norm_id.nii.gz')
mris = sorted(glob(os.path.join(dataset, "MRI", "*_id.nii.gz")))
mris_warped = sorted(glob(os.path.join(dataset, "MRI", "*_affine_deformable.nii.gz")))
warps = sorted(glob(os.path.join(dataset, "MRI", "*_affine_warp.nii.gz")))
invs = sorted(glob(os.path.join(dataset, "MRI", "*_affine_warp_inverse.nii.gz")))
mats = sorted(glob(os.path.join(dataset, "MRI", "*_affine.mat")))

In [11]:
def get_warp_from_mat_file(warp_outname, ref, mat):
    jacobian_command = "antsApplyTransforms -d 3 -n Linear -i " + str(ref) + " -o [" + str(warp_outname) + ",1] -r " + str(ref) + " -t " + str(mat)
    subprocess.call(jacobian_command.split(" "))

In [12]:
for i, mat in enumerate(mats):
    name = mris[i].split('/')[-1].split('.')[0].split('_')
    name = name[0] + '_' + name[1] + '_'
    outname = os.path.join(output_dir, "AffineWarp", name + "affinewarp.nii.gz")
    get_warp_from_mat_file(outname, atlas, mat)

In [13]:
for i, warp in enumerate(warps):
    name = mris[i].split('/')[-1].split('.')[0].split('_')
    name = name[0] + '_' + name[1] + '_'
    outname = os.path.join(output_dir, "DeformableWarp", name + "deformablewarp.nii.gz")
    shutil.copyfile(warp, outname)

In [14]:
def getLog10Jacobian_deformable_ants(warp_img, outfolder):
    outdir = os.path.join('output', 'GIN', outfolder, 'Log10Jacobian_Deformable')
    outname = outdir + '/Log10JacobianDeformable_' + warp_img.split('/')[-1].split('Warp_')[-1] + '.nii.gz'
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))

In [15]:
def getLog10Jacobian_affine_ants(warp_img, outfolder):
    outdir = os.path.join('output', 'GIN', outfolder, 'Log10Jacobian_Affine')
    outname = outdir + '/Log10JacobianAffine_' + warp_img.split('/')[-1].split('Warp_')[-1] + '.nii.gz'
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))

In [16]:
def getLog10Jacobian_affinedeformable_ants(aff_log10jac, def_log10jac, outfolder):
    outdir = os.path.join('output', 'GIN', outfolder, 'Log10Jacobian_AffineDeformable')
    outname = outdir + '/Log10JacobianAffineDeformable' + aff_log10jac.split('/')[-1].split('Log10JacobianAffine')[-1]
    combine_command = "ImageMath 3 " + str(outname) + " + " + str(aff_log10jac) + " " + str(def_log10jac)
    subprocess.call(combine_command.split(" "))

In [4]:
def getJacobian_deformable_ants(warp_img, outfolder):
    outdir = os.path.join('output', 'GIN', outfolder, 'Jacobian_Deformable')
    outname = outdir + '/DetJac_' + warp_img.split('/')[-1].split('Warp_')[-1] + '.nii.gz'
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 0 1"
    subprocess.call(jacobian_command.split(" "))

In [5]:
mridir = 'output/GIN/ANTS-Mean/'
affine_warps = sorted(glob(os.path.join(mridir, 'AffineWarp', "*.nii.gz")))
deformable_warps = sorted(glob(os.path.join(mridir, 'DeformableWarp', "*.nii.gz")))
#for i in range(len(affine_warps)):
    #getLog10Jacobian_affine_ants(affine_warps[i], outfolder)    
for i in range(len(deformable_warps)):
    #getLog10Jacobian_deformable_ants(deformable_warps[i], outfolder)  
    getJacobian_deformable_ants(deformable_warps[i], outfolder)

In [19]:
aff_log10jacs = sorted(glob(os.path.join(mridir, 'Log10Jacobian_Affine', "*.nii.gz")))
def_log10jacs = sorted(glob(os.path.join(mridir, 'Log10Jacobian_Deformable', "*.nii.gz")))
for i in range(len(aff_log10jacs)):
    getLog10Jacobian_affinedeformable_ants(aff_log10jacs[i], def_log10jacs[i], outfolder)

old stuff below

In [5]:
def getLog10Jacobian_deformable_ants(warp_img):
    outdir = os.path.join('output', 'GIN', 'ANTS-Mean', 'Log10Jacobian_Deformable')
    outname = outdir + '/Log10JacobianDeformable_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))

In [6]:
def getLog10Jacobian_affinedeformable_ants(warp_img, mat_img):
    outdir = os.path.join('output', 'GIN', 'ANTS-Mean', 'Log10Jacobian_AffineDeformable')
    outname = outdir + '/Log10JacobianAffineDeformable_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    combine_command = "ComposeMultiTransform 3 tmpWarp.nii.gz -R dataset2/Atlas/Norm_Atlas_MRI_128.nii.gz " + str(warp_img) + " " + str(mat_img)
    subprocess.call(combine_command.split(" "))
    jacobian_command = "CreateJacobianDeterminantImage 3 tmpWarp.nii.gz " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))

In [7]:
for i in range(len(warp_deformable)):
    getLog10Jacobian_deformable_ants(warp_deformable[i])    
    getLog10Jacobian_affinedeformable_ants(warp_deformable[i], mats_deformable[i])

output_image_filename: tmpWarp.nii.gz
reference_image_filename: dataset2/Atlas/Norm_Atlas_MRI_128.nii.gz
[0/2]: FIELD: dataset2/Feminad/MRI_N4_Resample_Norm_Deformable/Deformable_Norm_Resampled_N4_12_651701Warp.nii.gz
[1/2]: AFFINE: dataset2/Feminad/MRI_N4_Resample_Norm_Deformable/Deformable_Norm_Resampled_N4_12_651700GenericAffine.mat
output size: [128, 128, 128]
output spacing: [0.12, 0.12, 0.12]
output extension is: .gz
output_image_filename: tmpWarp.nii.gz
reference_image_filename: dataset2/Atlas/Norm_Atlas_MRI_128.nii.gz
[0/2]: FIELD: dataset2/Feminad/MRI_N4_Resample_Norm_Deformable/Deformable_Norm_Resampled_N4_12_651811Warp.nii.gz
[1/2]: AFFINE: dataset2/Feminad/MRI_N4_Resample_Norm_Deformable/Deformable_Norm_Resampled_N4_12_651810GenericAffine.mat
output size: [128, 128, 128]
output spacing: [0.12, 0.12, 0.12]
output extension is: .gz
output_image_filename: tmpWarp.nii.gz
reference_image_filename: dataset2/Atlas/Norm_Atlas_MRI_128.nii.gz
[0/2]: FIELD: dataset2/Feminad/MRI_N4_Res

output extension is: .gz
output_image_filename: tmpWarp.nii.gz
reference_image_filename: dataset2/Atlas/Norm_Atlas_MRI_128.nii.gz
[0/2]: FIELD: dataset2/Feminad/MRI_N4_Resample_Norm_Deformable/Deformable_Norm_Resampled_N4_2_6523201Warp.nii.gz
[1/2]: AFFINE: dataset2/Feminad/MRI_N4_Resample_Norm_Deformable/Deformable_Norm_Resampled_N4_2_6523200GenericAffine.mat
output size: [128, 128, 128]
output spacing: [0.12, 0.12, 0.12]
output extension is: .gz
output_image_filename: tmpWarp.nii.gz
reference_image_filename: dataset2/Atlas/Norm_Atlas_MRI_128.nii.gz
[0/2]: FIELD: dataset2/Feminad/MRI_N4_Resample_Norm_Deformable/Deformable_Norm_Resampled_N4_2_6524211Warp.nii.gz
[1/2]: AFFINE: dataset2/Feminad/MRI_N4_Resample_Norm_Deformable/Deformable_Norm_Resampled_N4_2_6524210GenericAffine.mat
output size: [128, 128, 128]
output spacing: [0.12, 0.12, 0.12]
output extension is: .gz
output_image_filename: tmpWarp.nii.gz
reference_image_filename: dataset2/Atlas/Norm_Atlas_MRI_128.nii.gz
[0/2]: FIELD: da

ConvertTransformFile 2 output0GenericAffine.mat temp.txt

edit temp.txt then use that in antsApplyTransforms

In [11]:
#def getLog10Jacobian_sitk(warp_img):
#    ddf = nib.load(warp_img).get_fdata().squeeze()
#    outdir = os.path.join('output', 'Feminad', 'ANTS', 'Log10Jacobian_sitk')
#    outname = 'Log10Jacobian_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
#    #itk_ddf = np.transpose(ddf, (1, 2, 3, 0))
#    itk_ddf = scipy.ndimage.gaussian_filter(ddf, sigma=1.0)
#    sitk_displacement_field = sitk.GetImageFromArray(itk_ddf, isVector=True)
#    jacobian_det_volume = sitk.DisplacementFieldJacobianDeterminant(sitk_displacement_field)
#    jacobian_det_np_arr = sitk.GetArrayViewFromImage(jacobian_det_volume)
#    #mri_inverse_transform = getRegistrationEvalInverseTransformForMRI(atlas, outdir + '/' + outname, atlas, save=True, mask=False)
#    #resample_jacobian = mri_inverse_transform(jacobian_det_np_arr).get_fdata()     
#    if(np.min(resample_jacobian) < 1e-5):
#        resample_jacobian += 1e-4   
#    return np.log10(resample_jacobian)

In [14]:
txt = 'output/Feminad/ANTS/Warp.txt'
for i in range(len(warp_deformable)):
    getLog10Jacobian_affine(warp_deformable[i], txt)
    break

output/Feminad/ANTS/test/Log10Jacobian_test_Deformable_Norm_Resampled_N4_12_651701.nii.gz
output_image_filename: output/Feminad/ANTS/AffineDeformable.nii.gz
reference_image_filename: output/Feminad/ANTS/Norm_Atlas_MRI_128.nii.gz
[0/2]: FIELD: dataset2/Feminad/MRI_N4_Resample_Norm_Deformable/Deformable_Norm_Resampled_N4_12_651701Warp.nii.gz
[1/2]: AFFINE: output/Feminad/ANTS/Warp.txt
output size: [128, 128, 128]
output spacing: [0.12, 0.12, 0.12]
output extension is: .gz
output_image_filename: output/Feminad/ANTS/Deformable.nii.gz
reference_image_filename: output/Feminad/ANTS/Norm_Atlas_MRI_128.nii.gz
[0/1]: FIELD: dataset2/Feminad/MRI_N4_Resample_Norm_Deformable/Deformable_Norm_Resampled_N4_12_651701Warp.nii.gz
output size: [128, 128, 128]
output spacing: [0.12, 0.12, 0.12]
output extension is: .gz


In [13]:
ataset2/Atlas/Norm_Atlas_MRI_128.nii.gzdef getLog10Jacobian_affine(warp, txt):
    outdir = os.path.join('output', 'Feminad', 'ANTS', 'test')    
    outname = outdir + '/Log10Jacobian_test_' + warp.split('/')[-1].split('Warp')[0] + '.nii.gz'
    print(outname)
    test_cmd = "ComposeMultiTransform 3 output/Feminad/ANTS/AffineDeformable.nii.gz -R output/Feminad/ANTS/Norm_Atlas_MRI_128.nii.gz " + str(warp) + " " + str(txt)
    subprocess.call(test_cmd.split(" "))
    test_cmd = "ComposeMultiTransform 3 output/Feminad/ANTS/Deformable.nii.gz -R output/Feminad/ANTS/Norm_Atlas_MRI_128.nii.gz " + str(warp)
    subprocess.call(test_cmd.split(" "))
    test_cmd = "CreateJacobianDeterminantImage 3 output/Feminad/ANTS/AffineDeformable.nii.gz output/Feminad/ANTS/Log10Jac_AffineDeformable.nii.gz 1 1"
    subprocess.call(test_cmd.split(" "))
    test_cmd = "CreateJacobianDeterminantImage 3 output/Feminad/ANTS/Deformable.nii.gz output/Feminad/ANTS/Log10Jac_Deformable.nii.gz 1 1"
    subprocess.call(test_cmd.split(" "))
    test_cmd = "ImageMath 3 output/Feminad/ANTS/Log10Jac_Affine.nii.gz - output/Feminad/ANTS/Log10Jac_AffineDeformable.nii.gz output/Feminad/ANTS/Log10Jac_Deformable.nii.gz"
    subprocess.call(test_cmd.split(" "))